In [1]:
import tensorflow as tf
import tensorboard.plugins.beholder as beholder_lib
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import os
import sys
from datetime import datetime

In [2]:
cur = os.path.dirname(os.path.realpath('beholder.ipynb'))
log_dir = os.path.join(cur,'logs')
print(log_dir)

/Users/EstherChang/Documents/Version_Control/Tensorboard/logs


In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.reshape(x_train, [-1,784])
x_test = np.reshape(x_test, [-1,784])
batch_size = 100
batch = len(x_train)//batch_size + 1
print("num of batch: %s" %batch)

num of batch: 601


In [4]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [5]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name = 'x-input')
    y_ = tf.placeholder(tf.int64, [None], name = 'y-input')
with tf.name_scope('input-reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)
def weight_variable(shape):
    weight = tf.Variable(tf.truncated_normal(shape, stddev = 0.1))
    return weight
def bias_variable(shape):
    bias = tf.Variable(tf.constant(0.1, shape=shape))
    return bias
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('hist', var)
def nn_layer(input_tensor, input_dim, output_dim, layer_name, activation=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weight'):
            weight = weight_variable([input_dim, output_dim])
            variable_summaries(weight)
        with tf.name_scope('bias'):
            bias = bias_variable([output_dim])
            variable_summaries(bias)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weight) + bias
            tf.summary.histogram('preactivate', preactivate)
        
        activations = activation(preactivate, name = 'activation')
        tf.summary.histogram('activation', activations)
    return activations

In [6]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('keep_prob', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y = nn_layer(dropped, 500, 10, 'layer2', activation=tf.identity)

with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
        cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels = y_, logits = y)
    tf.summary.scalar('cross_entropy', cross_entropy)

learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    #train_step = optimizer.minimize(cross_entropy)
    gradients, train_step = beholder_lib.Beholder.gradient_helper(optimizer, cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_pred = tf.equal(tf.argmax(y, 1), y_)
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy', accuracy)

merged_summary = tf.summary.merge_all()
time = datetime.now().strftime('%d-%H-%M-%S')
train_writer = tf.summary.FileWriter(os.path.join(log_dir,'train',time), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(log_dir,'test',time))
tf.global_variables_initializer().run()

In [7]:
beholder = beholder_lib.Beholder(logdir=log_dir)

In [ ]:
epoch = 500
for i in range(epoch):
    
    if i % 10 == 0:
        summary, acc, gradient_arrays = sess.run([merged_summary, accuracy, gradients ], 
                                feed_dict = {x:x_test, y_: y_test, keep_prob : 1.0})
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' %(i, acc))
    else:
        summary, gradient_arrays, _ = sess.run([merged_summary, gradients, train_step], 
                              feed_dict = {x:x_train[i*batch_size:(i+1)*batch_size]
                                           , y_: y_train[i*batch_size:(i+1)*batch_size], keep_prob : 0.8} )
        train_writer.add_summary(summary, i)
    beholder.update(session=sess, arrays=gradient_arrays)

Accuracy at step 0: 0.1214


In [ ]:
train_writer.close()
test_writer.close()
sess.close()